In [1]:
import pandas as pd 
import json 
import shapefile
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import os 
import json
import numpy as np

In [4]:
def isNested(entry): 
    try: 
        sum(entry)
        return 0 
    except: 
        return 1

def readInJson(fileName):
    try: 
        df = pd.read_json(fileName)
    except: 
        handle = json.load(open(fileName))
        df = pd.DataFrame(handle['features'])
        
    return df

lonlatSuburb = pd.read_csv('lonlatSuburb.csv')
vicPostcodes = pd.read_csv('vicPostcodes.csv')

lonlatSuburb['lat'] = lonlatSuburb.lat.apply(lambda x: round(x, 6))
lonlatSuburb['long'] = lonlatSuburb.long.apply(lambda x: round(x, 6))

vicPostcodes['lat'] = vicPostcodes.lat.apply(lambda x: round(x, 6))
vicPostcodes['long'] = vicPostcodes.long.apply(lambda x: round(x, 6))

In [17]:
garbageCollectionDate = sorted([i for i in os.listdir() if i.endswith('json')])

garbageCollectionDate[11]

'PortPhillip-garbage-collection-zones.geojson'

In [6]:
readInJson(garbageCollectionDate[10]).head(2)

,type,id,geometry,geometry_name,properties
0,Feature,ckan_f5d810f3_9b62_43c9_9a4e_e3afda01fe62.1,"{'type': 'MultiPolygon', 'coordinates': [[[[14...",geom,"{'linkfield': 'Halls Gap Monday', 'route': 'Ha..."
1,Feature,ckan_f5d810f3_9b62_43c9_9a4e_e3afda01fe62.2,"{'type': 'MultiPolygon', 'coordinates': [[[[14...",geom,"{'linkfield': 'Rural Monday', 'route': 'Rural'..."


In [7]:
df = readInJson(garbageCollectionDate[10])
df['geo'] = df.geometry.apply(lambda x: x['coordinates'][0][0])

df = df.explode('geo')

df['geo_nested'] = df.geo.apply(lambda x: isNested(x))
df['lat'] = df.geo.apply(lambda x: round(x[0], 6))
df['long'] = df.geo.apply(lambda x: round(x[1], 6))

df.drop(['geo','id', 'geometry', 'geometry_name',
         'geo_nested', 'type'], axis = 1, inplace = True)

In [9]:
df.iloc[0,0]

{'linkfield': 'Halls Gap Monday',
 'route': 'Halls Gap',
 'rub_day': 'Monday',
 'style': None,
 'id': 0,
 'rub_week': 1,
 'rub_start': '2018-11-19',
 'rec_day': 'Monday',
 'rec_week': 2,
 'rec_start': '2018-11-12',
 'info_url': 'https://www.ngshire.vic.gov.au/Council-Services/Bins-recycling-and-waste'}

In [10]:
ks = list(df.iloc[0,0].keys())
for i in range(df.shape[0]): 
    na = [x for x in list(df.iloc[i,0]) if x not in ks]
    if na: 
        print(na)

In [11]:
def collectionFrequency(row, wasteType): 
    
    rub_colDay = row[f'{wasteType}_day']
    rub_colFreq = row[f'{wasteType}_week']
    
    if rub_colFreq == '2' or rub_colFreq == 2: 
        rub_colFreq = 'Every Second '
    else: 
        rub_colFreq = 'Every '
    
    return rub_colFreq + rub_colDay

df['landfill_colDay'] = df.properties.apply(lambda x: collectionFrequency(x, 'rub'))
df['recyclable_colDay'] = df.properties.apply(lambda x: collectionFrequency(x, 'rec'))

df.drop('properties', axis = 1, inplace = True)

In [16]:
northerGrapians = lonlatSuburb.merge(df)[['locality', 'long', 'lat', 'landfill_colDay', 'recyclable_colDay']]\
.sort_values(['locality', 'landfill_colDay', 'recyclable_colDay'])\
        .drop_duplicates(['locality','recyclable_colDay', 'landfill_colDay', 'recyclable_colDay'])\
        .sort_values(['locality'])\
.merge(vicPostcodes[['locality', 'postcode']])\
.sort_values(['postcode', 'landfill_colDay', 'recyclable_colDay'])\
.drop_duplicates(subset = ['postcode', 'landfill_colDay', 'recyclable_colDay'])\
.rename(columns = dict(zip(['landfill_colDay', 'landfill_colDay', 'recyclable_colDay'], 
                           ['landfill', 'landfill', 'recyclable'])))\
.drop(['long', 'lat', 'locality'], axis = 1)\
.melt(id_vars = 'postcode')\
.rename(columns = {'variable':'binInfo_bin_type', 'value':'collection_time'})\
.drop_duplicates(['postcode', 'binInfo_bin_type'])

northerGrapians.to_csv('../GBD_northerGrapians.csv', index = False)